### 셋업

In [8]:
! pip3 install nltk

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kihyunwon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/kihyunwon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kihyunwon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 텍스트 전처리

In [40]:
text = "Nvidia's Huang says nuclear power an option to feed data warehouse"

In [41]:
tokens = text.split()
tokens

["Nvidia's",
 'Huang',
 'says',
 'nuclear',
 'power',
 'an',
 'option',
 'to',
 'feed',
 'data',
 'warehouse']

In [15]:
# NLTK 이용 토큰화
from nltk.tokenize import word_tokenize
word_tokens = word_tokenize(text)
word_tokens

['Nvidia',
 "'s",
 'Huang',
 'says',
 'nuclear',
 'power',
 'an',
 'option',
 'to',
 'feed',
 'data',
 'warehouse']

In [16]:
# 불용어 필터링
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_words = [w for w in word_tokens if not w.lower() in stop_words]
filtered_words

['Nvidia',
 "'s",
 'Huang',
 'says',
 'nuclear',
 'power',
 'option',
 'feed',
 'data',
 'warehouse']

In [17]:
# 어간 추출
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in filtered_words]
stemmed_words

['nvidia',
 "'s",
 'huang',
 'say',
 'nuclear',
 'power',
 'option',
 'feed',
 'data',
 'warehous']

In [18]:
# 기본형 추출
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
lemmatized_words

['Nvidia',
 "'s",
 'Huang',
 'say',
 'nuclear',
 'power',
 'option',
 'feed',
 'data',
 'warehouse']

### Batch Index Pipeline

### 셋업

In [1]:
# Skip if you already have Java

# # Install Homebrew
# /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# # Install OpenJDK
# brew install openjdk@17

# # set JAVA_HOME in zshrc
# export JAVA_HOME=$(/usr/libexec/java_home -v17)
# export PATH=$JAVA_HOME/bin:$PATH

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 34.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840651 sha256=63b8c5cdebc9ab3f6dac5a31e12873f07e0a10b12f5b0c722343c4883b703afa
  Stored in directory: /Users/kihyunwon/Library/Caches/pip/wheels/2e/d2/18/6f4f20e8332359f7fffceb6828edcc80ef96f86744192a7bb9
Successfully built pyspark


In [ ]:
# Skip if you already have Spark

# # Download and install Spark
# ! curl -O https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
# ! tar -xzf spark-3.4.3-bin-hadoop3.tgz
# # Move to /opt folder
# ! sudo mv spark-3.4.3-bin-hadoop3 /opt/spark-3.4.3
# cp /opt/spark-3.4.3/conf/spark-defaults.conf.template /opt/spark-3.4.3/conf/spark-defaults.conf
# echo 'spark.driver.extraJavaOptions   -Djava.security.manager=allow' >> /opt/spark-3.4.3/conf/spark-defaults.conf
# echo 'spark.executor.extraJavaOptions   -Djava.security.manager=allow' >> /opt/spark-3.4.3/conf/spark-defaults.conf
# # Create symlink
# ! sudo ln -s /opt/spark-3.4.3 /opt/spark

# # set SPARK_HOME in .zshrc
# export SPARK_HOME=/opt/spark
# export PATH=$SPARK_HOME/bin:$PATH

# # Jupyter Notebook Spark integration
# export PYSPARK_PYTHON=$(brew --prefix python)/libexec/bin/python
# export PYSPARK_DRIVER_PYTHON='jupyter'
# export PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port=8889'
# # Download elastic-spark jar from https://central.sonatype.com/artifact/org.elasticsearch/elasticsearch-spark-30_2.12/versions
# export PYSPARK_SUBMIT_ARGS='--jars /opt/spark-3.4.3/jars/elasticsearch-spark-30_2.12-8.15.0.jar pyspark-shell'

In [ ]:
# Skip if you already have Elasticsearch

# Download Docker from https://docs.docker.com/engine/install/

# # pull docker images
# docker pull docker.elastic.co/elasticsearch/elasticsearch:8.15.0
# docker pull docker.elastic.co/kibana/kibana:8.15.0

# # start Elasticsearch and Kibana containers
# docker network create elastic
# docker run -d --name elasticsearch --net elastic -p 9200:9200 -p 9300:9300 -m 1GB -e "discovery.type=single-node" -e "ELASTIC_PASSWORD=password" -e "xpack.security.enabled=false" -e "xpack.security.enrollment.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.15.0
# docker run -d --name kibana --net elastic -p 5601:5601 docker.elastic.co/kibana/kibana:8.15.0

In [23]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf()
spark = SparkSession.builder.appName("indexer").config(conf=conf).getOrCreate()

# Dataset: https://www.kaggle.com/datasets/ahsenwaheed/youtube-comments-spam-dataset?resource=download
path = "Youtube-Spam-Dataset.csv"

df = spark.read.csv(path)
df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|                 _c0|                 _c1|                _c2|                 _c3|                 _c4|  _c5|
+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|          COMMENT_ID|              AUTHOR|               DATE|             CONTENT|          VIDEO_NAME|CLASS|
|LZQPQhLyRh80UYxNu...|           Julius NM|2013-11-07T06:20:48|Huh, anyway check...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh_C2cTtd...|         adam riyati|2013-11-07T12:37:15|Hey guys check ou...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh9MSZYnf...|    Evgeny Murashkin|2013-11-08T17:34:21|just for test I h...|PSY - GANGNAM STY...|    1|
|z13jhp0bxqncu512g...|     ElNino Melendez|2013-11-09T08:28:43|me shaking my sex...|PSY - GANGNAM STY...|    1|
|z13fwbwp1oujthgqj...|              GsMega|2013-11-10T16:05:38|watch?v=vtaRGgvGt...|PSY - GANGNAM STY...

In [24]:
columns = df.columns
new_df = df.filter(df._c0 != 'COMMENT_ID')
new_df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+---+
|                 _c0|                 _c1|                _c2|                 _c3|                 _c4|_c5|
+--------------------+--------------------+-------------------+--------------------+--------------------+---+
|LZQPQhLyRh80UYxNu...|           Julius NM|2013-11-07T06:20:48|Huh, anyway check...|PSY - GANGNAM STY...|  1|
|LZQPQhLyRh_C2cTtd...|         adam riyati|2013-11-07T12:37:15|Hey guys check ou...|PSY - GANGNAM STY...|  1|
|LZQPQhLyRh9MSZYnf...|    Evgeny Murashkin|2013-11-08T17:34:21|just for test I h...|PSY - GANGNAM STY...|  1|
|z13jhp0bxqncu512g...|     ElNino Melendez|2013-11-09T08:28:43|me shaking my sex...|PSY - GANGNAM STY...|  1|
|z13fwbwp1oujthgqj...|              GsMega|2013-11-10T16:05:38|watch?v=vtaRGgvGt...|PSY - GANGNAM STY...|  1|
|LZQPQhLyRh9-wNRtl...|        Jason Haddad|2013-11-26T02:55:11|Hey, check out my...|PSY - GANGNAM STY...|  1|
|z13lfzdo5

In [25]:
df_schema = df.schema

new_columns = []
for i in df.limit(1).collect()[0]:
  new_columns.append(i)

for i,k in enumerate(df_schema.fields):
  k.name = new_columns[i]

# Apply the schema to the original dataframe
new_df = spark.createDataFrame(new_df.rdd, df_schema)
new_df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|          COMMENT_ID|              AUTHOR|               DATE|             CONTENT|          VIDEO_NAME|CLASS|
+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|LZQPQhLyRh80UYxNu...|           Julius NM|2013-11-07T06:20:48|Huh, anyway check...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh_C2cTtd...|         adam riyati|2013-11-07T12:37:15|Hey guys check ou...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh9MSZYnf...|    Evgeny Murashkin|2013-11-08T17:34:21|just for test I h...|PSY - GANGNAM STY...|    1|
|z13jhp0bxqncu512g...|     ElNino Melendez|2013-11-09T08:28:43|me shaking my sex...|PSY - GANGNAM STY...|    1|
|z13fwbwp1oujthgqj...|              GsMega|2013-11-10T16:05:38|watch?v=vtaRGgvGt...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh9-wNRtl...|        Jason Haddad|2013-11-26T02:55:11|Hey, check out my...|PSY - GANGNAM STY...

In [29]:
# Write the DataFrame to an Elasticsearch index
es_conf = {
    "es.nodes.discovery": "false",
    "es.nodes.data.only": "false",
    "es.net.http.auth.user": "elastic",
    "es.net.http.auth.pass": "password",
    "es.index.auto.create": "true",
    "es.nodes": "http://127.0.0.1",
    "es.port": "9200",
    "es.mapping.id": "COMMENT_ID",
}

new_df.write.mode("append") \
        .format('org.elasticsearch.spark.sql') \
        .options(**es_conf) \
        .save("youtube")

24/10/04 22:26:23 ERROR NetworkClient: Node [172.18.0.2:9200] failed (org.elasticsearch.hadoop.thirdparty.apache.commons.httpclient.ConnectTimeoutException: The host did not accept the connection within timeout of 60000 ms); selected next node [127.0.0.1:9200]


In [36]:
# Verify the index in Kibana http://localhost:5601/app/dev_tools#/console
! curl -X GET http://localhost:9200/youtube/_search\?q\=MB%20LOVE

{"took":7,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":579,"relation":"eq"},"max_score":12.020087,"hits":[{"_index":"youtube","_id":"z123std54m2ozht10232efr5svb4vh0au04","_score":12.020087,"_source":{"COMMENT_ID":"z123std54m2ozht10232efr5svb4vh0au04","AUTHOR":"MB LOVE","DATE":"2014-08-13T03:04:34","CONTENT":"damn, nvm what I said","VIDEO_NAME":"Katy Perry - Roar","CLASS":"1"}},{"_index":"youtube","_id":"z13rzhjaxxr3z1t4504ccvlp1uqnddvqoyg0k","_score":5.402604,"_source":{"COMMENT_ID":"z13rzhjaxxr3z1t4504ccvlp1uqnddvqoyg0k","AUTHOR":"Kate Love","DATE":"2015-05-23T01:14:31.812000","CONTENT":"Best song﻿","VIDEO_NAME":"Eminem - Love The Way You Lie ft. Rihanna","CLASS":"0"}},{"_index":"youtube","_id":"_2viQ_Qnc6_onwOgxju-DV6WkqHZEOztCXD04EgEFBU","_score":5.402604,"_source":{"COMMENT_ID":"_2viQ_Qnc6_onwOgxju-DV6WkqHZEOztCXD04EgEFBU","AUTHOR":"Hidden Love","DATE":"2013-09-24T14:29:54.072000","CONTENT":"Hi. Check out and share our songs

### Streaming Index Pipeline

In [3]:
# Skip if you already have Kafka

# # pull docker images
# ! docker pull apache/kafka:3.8.0

# Start Kafka and Flink containers
# ! docker run -d --name kafka -p 9092:9092 apache/kafka:3.8.0

# # Install kafka-python client
# ! pip3 install kafka-python

# # Download spark-sql-kafka jar from https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.4.3
# # Download kafka-client jar from https://mvnrepository.com/artifact/org.apache.kafka/kafka-clients/3.3.2
# export PYSPARK_SUBMIT_ARGS='--jars /opt/spark-3.4.3/jars/spark-sql-kafka-0-10_2.12-3.4.3.jar,/opt/spark-3.4.3/jars/kafka-clients-3.3.2.jar pyspark-shell'

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [31]:
import json
import datetime

from kafka import KafkaProducer

def on_success(metadata):
    print(f"Message produced with the offset: {metadata.offset}")

def on_error(error):
    print(f"An error occurred while publishing the message. {error}")

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

message = {
    "COMMENT_ID" : "z123std54m2ozht10232efr5svb4vh0au04",
    "CONTENT": "damn nvm what I said"
}

# Send updates to 'youtube_update' topic
future = producer.send("youtube_update", message)
future.add_callback(on_success)
future.add_errback(on_error)

# Ensure all messages are sent before exiting
producer.flush()
producer.close()

Message produced with the offset: 6


In [32]:
# Download profane words dataset: https://www.kaggle.com/datasets/konradb/profanities-in-english-collection
path = "profanity_en.csv"
df = spark.read.csv(path)
profane_df = df.select(df._c1)
profane_df = profane_df.filter(df._c1 != 'canonical_form_1')
profane_df.show()

profane_set = set(profane_df.select(profane_df._c1).rdd.flatMap(lambda x: x).collect())

+-----------+
|        _c1|
+-----------+
|         69|
|        ass|
|       fuck|
|       fuck|
|       fuck|
|        ass|
|        sex|
|        sex|
|     orgasm|
|     orgasm|
|ejaculation|
|       arse|
|       arse|
|       arse|
|   foreskin|
|       shit|
|       shit|
|      skank|
|        ass|
|    abraham|
+-----------+
only showing top 20 rows



In [33]:
# Read from kafka and mask profane words from content
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("COMMENT_ID", StringType(), True),
    StructField("CONTENT", StringType(), True)
])

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "youtube_update") \
    .option("startingOffsets", "latest") \
    .load()

# data = {
#     "COMMENT_ID" : "z123std54m2ozht10232efr5svb4vh0au04",
#     "CONTENT": "damn nvm what I said"
# }

json_stream = kafka_stream.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema).alias("data")) \
    .select("data.*")

In [42]:
# Update the DataFrame to an Elasticsearch index
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

es_conf = {
    "es.nodes.discovery": "false",
    "es.nodes.data.only": "false",
    "es.net.http.auth.user": "elastic",
    "es.net.http.auth.pass": "password",
    "es.nodes": "http://127.0.0.1",
    "es.port": "9200",
    "es.mapping.id": "COMMENT_ID",
    "es.mapping.exclude": "COMMENT_ID",
    "es.write.operation": "update",
    "checkpointLocation": "/tmp/",
    "es.spark.sql.streaming.sink.log.enabled": "false",
}

def mask_profane_word(s):
    words = s.split()
    tokens = [t if t not in profane_set else '****' for t in words]
    return ' '.join(tokens)

def mask_profane_word_df(df, epoch_id):
    name = 'CONTENT'
    udf = UserDefinedFunction(lambda x: mask_profane_word(x), StringType())
    new_df = df.select(*[udf(column).alias(name) if column == name else column for column in df.columns])
    
    new_df.write.mode("append") \
        .format('org.elasticsearch.spark.sql') \
        .options(**es_conf) \
        .save('youtube')
   
query = json_stream.writeStream \
        .outputMode("append") \
        .foreachBatch(mask_profane_word_df) \
        .options(**es_conf) \
        .start("youtube")

query.awaitTermination()

24/10/05 01:23:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/10/05 01:23:56 WARN StreamingQueryManager: Stopping existing streaming query [id=2390ed90-ff92-430c-a198-29e5e13fb07b, runId=c62c7339-749a-4c07-b1ab-913b7c9ba947], as a new run is being started.
24/10/05 01:23:56 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/homebrew/Cellar/python@3.9/3.9.20/Frameworks/Python.fra

KeyboardInterrupt: 

In [37]:
! curl -X GET http://localhost:9200/youtube/_search\?q\=MB%20LOVE

{"took":17,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":579,"relation":"eq"},"max_score":12.020087,"hits":[{"_index":"youtube","_id":"z123std54m2ozht10232efr5svb4vh0au04","_score":12.020087,"_source":{"COMMENT_ID":"z123std54m2ozht10232efr5svb4vh0au04","AUTHOR":"MB LOVE","DATE":"2014-08-13T03:04:34","CONTENT":"damn, nvm what I said","VIDEO_NAME":"Katy Perry - Roar","CLASS":"1"}},{"_index":"youtube","_id":"z13rzhjaxxr3z1t4504ccvlp1uqnddvqoyg0k","_score":5.402604,"_source":{"COMMENT_ID":"z13rzhjaxxr3z1t4504ccvlp1uqnddvqoyg0k","AUTHOR":"Kate Love","DATE":"2015-05-23T01:14:31.812000","CONTENT":"Best song﻿","VIDEO_NAME":"Eminem - Love The Way You Lie ft. Rihanna","CLASS":"0"}},{"_index":"youtube","_id":"_2viQ_Qnc6_onwOgxju-DV6WkqHZEOztCXD04EgEFBU","_score":5.402604,"_source":{"COMMENT_ID":"_2viQ_Qnc6_onwOgxju-DV6WkqHZEOztCXD04EgEFBU","AUTHOR":"Hidden Love","DATE":"2013-09-24T14:29:54.072000","CONTENT":"Hi. Check out and share our song